# Preparation

In [1]:
"""
# Download model
huggingface-cli download tiiuae/Falcon3-1B-Instruct-1.58bit --local-dir ~/models/tiiuae/Falcon3-1B-Instruct-1.58bit
# Compile
python setup_env.py -md ~/models/tiiuae/Falcon3-1B-Instruct-1.58bit -q i2_s

# Download Falcon3-1B-Instruct
huggingface-cli download tiiuae/Falcon3-1B-Instruct --local-dir ~/models/tiiuae/Falcon3-1B-Instruct
"""

'\n# Download model\nhuggingface-cli download tiiuae/Falcon3-1B-Instruct-1.58bit --local-dir ~/models/tiiuae/Falcon3-1B-Instruct-1.58bit\n# Compile\npython setup_env.py -md ~/models/tiiuae/Falcon3-1B-Instruct-1.58bit -q i2_s\n\n# Download Falcon3-1B-Instruct\nhuggingface-cli download tiiuae/Falcon3-1B-Instruct --local-dir ~/models/tiiuae/Falcon3-1B-Instruct\n'

# Importing Libraries

In [2]:
import os
from dataclasses import dataclass
import torch
from transformers import AutoTokenizer
from utils import set_seed, generate_response
from utils import BitNet
#from speculative_decoding import generate_draft_response, speculative_decoding, verify_with_target

/home/pathfinder/miniconda3/envs/bitnet-cpp/lib/python3.9/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


# Configuration

In [3]:
@dataclass
class CONFIG:
    # Debug
    debug: bool = False
    verbose: bool = True

    # Model
    tokenizer_id: str = "tiiuae/Falcon3-1B-Instruct"
    model_path: str = "/home/pathfinder/models/tiiuae/Falcon3-1B-Instruct-1.58bit/ggml-model-f32.gguf"
    quantized_path: str = "/home/pathfinder/models/tiiuae/Falcon3-1B-Instruct-1.58bit/ggml-model-i2_s.gguf"
    ctx_size: int = 1024

    # Generation
    max_new_tokens: int = 256
    ## Speculative Decoding
    num_assistant_tokens: int = 5
    assistant_confidence_threshold: float = 0.4

    # Device
    n_threads: int = 12

    # Seed
    seed = 42

config = CONFIG()

In [4]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = "cpu"
eos_token = "<|endoftext|>"
pad_token = "<|pad|>"
assistant_token = "<|assistant|>"

In [5]:
set_seed(config.seed)

Random seed set to 42


In [6]:
system_prompt = "You are an helpful assistant."
user_prompt = "Explain quantum mechanics in detail please."
assistant_response = "Quantum mechanics is a fundamental theory in physics that describes the physical properties of nature at the scale of atoms and subatomic particles, such as electrons, protons, and photons."

# Model

In [7]:
tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_id, use_fast=True)
tokenizer.pad_token = pad_token

In [8]:
bitnet = BitNet(
    model_path=config.quantized_path,
    host="127.0.0.1",
    port=8080,
    ctx_size=config.ctx_size,
    n_threads=config.n_threads,
    n_gpu_layers=0,
    batch_size=1,
    slot_id=1
)
bitnet.start_server()

🚀 Starting llama-server on 127.0.0.1:8080
✅ Server is ready.


In [9]:
bitnet.generate(
    text=bitnet.format_falcon_prompt(
        tokenizer=tokenizer,
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        assistant_response=assistant_response
    ),
    max_new_tokens=config.max_new_tokens,
    verbose=config.verbose
)


──────────────────────────────────────────────────
🧠 Generation Info
──────────────────────────────────────────────────
💬 User Input:
<|system|>
You are an helpful assistant.
<|user|>
Explain quantum mechanics in detail please.
<|assistant|>
Quantum mechanics is a fundamental theory in physics that describes the physical properties of nature at the scale of atoms and subatomic particles, such as electrons, protons, and photons.

🟢 Generated Text:
 At its most basic level, quantum mechanics is the result of different theories be a model of the quantum world. Three of the main theories according to this theory are: The quantum theory of the atom, the quantum theory of the proton, and the quantum theory of the radiation.

1) Quantum Theory of the Atom:

The quantum theory of the atom is based on the realization that light waves, such as electrons, are not a wave as was classically thought, but a particle. This thought was also recognized by Werner Heisenberg and Louis Lange. The quantum 

' At its most basic level, quantum mechanics is the result of different theories be a model of the quantum world. Three of the main theories according to this theory are: The quantum theory of the atom, the quantum theory of the proton, and the quantum theory of the radiation.\n\n1) Quantum Theory of the Atom:\n\nThe quantum theory of the atom is based on the realization that light waves, such as electrons, are not a wave as was classically thought, but a particle. This thought was also recognized by Werner Heisenberg and Louis Lange. The quantum theory of the atom describes the particle representation of electrons and positrons, as well as the electron-positron interaction.\n\nIn the answers to mechanics problems, the relationship between the different quantities in quantum mechanics is used to read the overall electron configuration of the atom. The two complementary properties of mass and charge and their electronic distribution can be described by methods called quantumconstants:\n

In [10]:
bitnet.verify(
    text=bitnet.format_falcon_prompt(
        tokenizer=tokenizer,
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        assistant_response=assistant_response
    ),
    num_verify=config.num_assistant_tokens,
    threshold=config.assistant_confidence_threshold,
    verbose=config.verbose
)

Error during verification API request: 501 Server Error: Not Implemented for url: http://127.0.0.1:8080/infill


{'cut_index': 0, 'probs': []}

In [11]:
asdf

NameError: name 'asdf' is not defined

In [ ]:
model = BitNet(
    model_path=config.quantized_path,
    host="127.0.0.2",
    port=8080,
    ctx_size=config.ctx_size,
    n_threads=config.n_threads,
    n_gpu_layers=0,
    batch_size=1,
    slot_id=1
)
model.start_server()

In [ ]:
asdf

In [ ]:
# C++ BitNet
run_server(
    model_path=config.quantized_path,
    ctx_size=config.n_ctx,
    n_threads=config.n_threads,
    verbose=config.verbose
)

In [ ]:
res = generate_draft_response(
    model_path=config.quantized_path,
    prompt=format_falcon_prompt(
        tokenizer=tokenizer,
        system_prompt=system_prompt,
        user_prompt=user_prompt
    ),
    ctx_size=config.n_ctx,
    n_threads=config.n_threads,
    max_new_tokens=config.max_new_tokens,
    seed=config.seed,
    verbose=config.verbose
)

In [ ]:
result = speculative_decoding(
    tokenizer=tokenizer,
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    target_model=model,
    max_new_tokens=config.max_new_tokens,
    num_assistant_tokens=config.num_assistant_tokens,
    confidence_threshold=config.assistant_confidence_threshold,
    seed=config.seed,
    verbose=True,
    server_url="http://localhost:8080"
)

In [ ]:
asdf

In [ ]:
quantized_model = AutoModelForCausalLM.from_pretrained(
    config.quantized_id,
    device_map=device,
    dtype=torch.bfloat16
)
quantized_model.eval()
print(quantized_model)
print(f"Number of parameters: {quantized_model.num_parameters() / 1e9:.2f}B")

# Generation

In [ ]:
if config.debug:
    print(format_falcon_prompt(
        tokenizer=tokenizer,
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        assistant_response=assistant_response
    ))

In [ ]:
generate_response(
    tokenizer=tokenizer,
    model=model,
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    max_new_tokens=config.max_new_tokens,
    #num_assistant_tokens=config.num_assistant_tokens,
    #assistant_confidence_threshold=config.assistant_confidence_threshold,
    verbose=config.verbose
)

In [ ]:
if config.debug:
    generate_response(
        tokenizer=tokenizer,
        model=quantized_model,
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        max_new_tokens=config.max_new_tokens,
        #num_assistant_tokens=config.num_assistant_tokens,
        #assistant_confidence_threshold=config.assistant_confidence_threshold,
        verbose=config.verbose
    )

## Speculative Decoding

In [ ]:
if config.debug:
    generate_response(
        tokenizer=tokenizer,
        model=model,
        assistant_model=quantized_model,
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        max_new_tokens=config.max_new_tokens,
        num_assistant_tokens=config.num_assistant_tokens,
        assistant_confidence_threshold=config.assistant_confidence_threshold,
        verbose=config.verbose
    )

# C++

## Speculative Decoding

In [ ]:
sample = generate_draft_response(
    model_path=config.quantized_path,
    prompt=format_falcon_prompt(
        tokenizer=tokenizer,
        system_prompt=system_prompt,
        user_prompt=user_prompt
    ),
    ctx_size=config.n_ctx,
    n_threads=config.n_threads,
    max_new_tokens=10,
    seed=config.seed,
    verbose=False
)
res = verify_with_target(
    context=sample,
    tokenizer=tokenizer,
    target_model=model,
    num_assistant_tokens=10,
    confidence_threshold=0.9,
    verbose=True
)

In [ ]:
result = speculative_decoding(
    tokenizer=tokenizer,
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    target_model=model,
    draft_model=config.quantized_path,
    ctx_size=config.n_ctx,
    n_threads=config.n_threads,
    max_new_tokens=config.max_new_tokens,
    num_assistant_tokens=config.num_assistant_tokens,
    confidence_threshold=config.assistant_confidence_threshold,
    seed=config.seed,
    verbose=True
)